# Example ticket generation workflow

This notebook sets up an environment, queries mediacloud for sources, creates Source objects and runs 'issues' on them, then generates tickets in zammad out of the detected issues

In [1]:
#%pip install -q -e  ../. 
import os

mc_token = "BYO"
os.environ["MC_API_TOKEN"] = mc_token
os.environ["ZAMMAD_TOKEN"] = "BYO"

from directory_issues import Collection, Source
from directory_issues.clients.zammad_client import ZammadClient
from IPython.display import HTML

zammad_token='BYO' zammad_url='https://support.mediacloud.org/api/v1/' default_zammad_group='Users' default_zammad_user='Directory Detective' default_zammad_email='support@mediacloud.org'


## Use the mediacloud directory to get a specific subset of sources
In this case, those that have a defined 'url search string'


In [2]:
import mediacloud
mc_dir = mediacloud.api.DirectoryApi(mc_token)
sources_with_url_search_strings = []

more_sources = True
offset = 0
limit=100000
while more_sources:
    
    resp = mc_dir.source_list(limit = limit, offset=offset)
    matches = [r for r in resp["results"] if r["url_search_string"] is not None]
    S = [Source(r) for r in matches]
    for s in S:
        print(s)
        s.find_issues()
    sources_with_url_search_strings.extend(S)
    print(len(sources_with_url_search_strings))
    if resp["next"] is not None:
        offset += limit
    else:
        more_sources = False
        

Source(indiatimes.com, http://timesofindia.indiatimes.com/)
Source(ctvnews.ca, http://bc.ctvnews.ca/)
Source(nation.africa, https://nation.africa/)
Source(theguardian.com, https://www.theguardian.com/us)
Source(intoday.in, http://indiatoday.intoday.in/#spider)
Source(cbslocal.com, https://www.cbsnews.com/boston/)
Source(forward.com, http://forward.com)
Source(bizjournals.com, http://www.bizjournals.com/boston)
Source(uol.com.br, http://www.folha.uol.com.br/)
Source(iol.co.za, http://www.iol.co.za/the-star)
Source(cbslocal.com, http://sanfrancisco.cbslocal.com)
Source(bizjournals.com, http://www.bizjournals.com/sanfrancisco/)
Source(bdlive.co.za, http://www.bdlive.co.za/)
Source(kenyanews.go.ke, https://www.kenyanews.go.ke/)
Source(ctvnews.ca, http://regina.ctvnews.ca/)
Source(ctvnews.ca, http://windsor.ctvnews.ca/)
Source(news12.com, http://connecticut.news12.com/)
Source(nation.africa, https://nairobinews.nation.africa/)
Source(cbslocal.com, http://newyork.cbslocal.com)
Source(india.c

## A demo source ticket with these issues


In [3]:
sources_with_issues = [s for s in sources_with_url_search_strings if len(s.source_issues) != 0]
source = sources_with_issues[10]
print(source.source_data.id)
print(source.source_data.label)
message = source.render_template()
HTML(message)


40943
ena


## Using the zamad client to create tickets 
the source class has a method wrapped inside for this task. 

In [4]:
len(sources_with_issues)

84

In [5]:
for source in sources_with_issues:
    pass
    #source.post_zammad_issue()

In [6]:
import requests

In [8]:
import os
print(os.environ.get('HTTP_PROXY'))
print(os.environ.get('HTTPS_PROXY'))
print(os.environ.get('NO_PROXY'))

http://127.0.0.1:8000
http://127.0.0.1:8000
None


In [9]:

os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)
os.environ.pop('NO_PROXY', None)


In [19]:
issue_map = {source:[i["issue_name"] for i in source.source_issues] for source in sources_with_issues}
import pprint
pprint.pprint(issue_map)

{Source(pravda.ru, http://english.pravda.ru#spider): ['USS-*-Postfix'],
 Source(coastweek.com, http://www.coastweek.com): ['USS-*-Prefix',
                                                   'USS-*-Postfix',
                                                   'Empty-USS'],
 Source(news12.com, http://westchester.news12.com/): ['USS-*-Postfix'],
 Source(eturbonews.com, http://www.eturbonews.com/#spider): ['USS-*-Postfix'],
 Source(bizjournals.com, http://upstart.bizjournals.com/#spider): ['USS-*-Postfix'],
 Source(yahoo.com, http://uk.news.yahoo.com/): ['USS-*-Postfix'],
 Source(nation.africa, https://nation.africa/): ['USS-*-Prefix',
                                                 'USS-*-Postfix',
                                                 'Empty-USS'],
 Source(wp.pl, http://wiadomosci.wp.pl): ['USS-*-Postfix'],
 Source(zenit.org, http://www.zenit.org/#spider): ['USS-*-Postfix'],
 Source(kenyanews.go.ke, https://www.kenyanews.go.ke/): ['USS-*-Prefix',
                              

In [24]:
postfixes = [s for s,i in issue_map.items() if 'USS-*-Postfix' in i and len(i) == 1]

In [26]:
for s in postfixes:
    print(s)
    print(s.source_data.url_search_string)
    fixed_uss = s.source_data.url_search_string + "/*"
    print(fixed_uss)

Source(people.com.cn, http://english.people.com.cn/)
english.people.com.cn
english.people.com.cn/*
Source(chinadaily.com.cn, http://usa.chinadaily.com.cn/#spider)
usa.chinadaily.com.cn
usa.chinadaily.com.cn/*
Source(cbslocal.com, http://gooddaysacramento.cbslocal.com)
gooddaysacramento.cbslocal.com
gooddaysacramento.cbslocal.com/*
Source(voanews.com, http://ir.voanews.com/)
ir.voanews.com
ir.voanews.com/*
Source(xinhuanet.com, http://www.xinhuanet.com/english/)
english.news.cn
english.news.cn/*
Source(vietnamplus.vn, http://en.vietnamplus.vn/)
en.vietnamplus.vn
en.vietnamplus.vn/*
Source(ena.et, https://www.ena.et/en/)
ena.et/en
ena.et/en/*
Source(euronews.com, http://www.euronews.com/#spider)
de.euronews.com
de.euronews.com/*
Source(indiatimes.com, https://economictimes.indiatimes.com/)
economictimes.indiatimes.com
economictimes.indiatimes.com/*
Source(censor.net.ua, http://censor.net.ua/#spider)
en.censor.net.ua
en.censor.net.ua/*
Source(ukranews.com, https://ukranews.com/)
ukranews.

In [41]:
ex = postfixes[-5]
print(ex)

Source(yahoo.com, http://uk.news.yahoo.com/)


In [59]:
#monkey-patch!?
## A quick but futile attempt at building a method to bulk-edit the source values.
## It's better that it doesn't work (csrf...) but still rather sad.
from types import MethodType

def patch_source_url_search_string(self, id, url_search_string):
    endpoint_url = self.BASE_API_URL + f"sources/sources/{id}"
    og = self.source(id)
    og["url_search_string"] = url_search_string
    

    response = self._session.patch(endpoint_url, json=og)
    print(response.status_code)

mc_dir.patch_source_url_search_string = MethodType(patch_source_url_search_string, mc_dir)

In [60]:
print(ex.source_data.id)
print(ex.source_data.url_search_string)

414183
uk.news.yahoo.com


In [61]:
mc_dir.patch_source_url_search_string(ex.source_data.id, ex.source_data.url_search_string+"/*")

403
